In [4]:
import geopypi.common as geopypi
urls = ["https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/NHD/State/GDB/NHD_H_District_of_Columbia_State_GDB.zip"]
geopypi.download_files(urls, out_dir='.', unzip=True)

Downloading...
From: https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/NHD/State/GDB/NHD_H_District_of_Columbia_State_GDB.zip
To: d:\spring 24\software design\Geopypi\docs\examples\NHD_H_District_of_Columbia_State_GDB.zip
100%|██████████| 73.5M/73.5M [00:17<00:00, 4.15MB/s]


Extracting files...
